In [ ]:
import tkinter
import customtkinter
import RPi.GPIO as GPIO
import time
from time import sleep, strftime, time
import spidev
import csv
import threading

customtkinter.set_appearance_mode("light")
customtkinter.set_default_color_theme("dark-blue")

GPIO.setwarnings(False)
GPIO.setmode(GPIO.BOARD)

Relay1 = 11
Relay2 = 15
Relay3 = 13
Relay4 = 22

GPIO.setup(Relay1, GPIO.OUT, initial=GPIO.LOW)
GPIO.setup(Relay2, GPIO.OUT, initial=GPIO.LOW)
GPIO.setup(Relay3, GPIO.OUT, initial=GPIO.LOW)
GPIO.setup(Relay4, GPIO.OUT, initial=GPIO.LOW)

BMS_alarm1 = None
BMS_alarm2 = None
SCADA_alarm1 = None
SCADA_alarm2 = None

current = None

log_status = False

spi = spidev.SpiDev()
spi.open(0, 0)  
spi.max_speed_hz = 1350000

focused_entry = None

def on_focus(entry):
    global focused_entry
    focused_entry = entry  

def BMS_set1():
    global BMS_alarm1
    if BMS_first_input.get() < 0:
        print("ERROR: INVALID VALUE")

    elif BMS_first_input.get() >= 0:
        BMS_alarm1 = BMS_first_input.get()
        print("BMS Alarm 1 set to:", BMS_alarm1)
        

def BMS_set2():
    global BMS_alarm2
    if BMS_second_input.get() <= BMS_first_input.get():
        print("The second alarm cannot be lower than the first alarm")
         
    elif BMS_second_input.get() > BMS_first_input.get():
        BMS_alarm2 = BMS_second_input.get()
        print("BMS Alarm 2 set to:", BMS_alarm2)

def SCADA_set1():
    global SCADA_alarm1
    if SCADA_first_input.get() < 0:
        print("ERROR: INVALID VALUE")

    elif SCADA_first_input.get() >= 0:
        SCADA_alarm1 = SCADA_first_input.get()
        print("BMS Alarm 1 set to:", SCADA_alarm1)

def SCADA_set2():
    global SCADA_alarm2
    if SCADA_second_input.get() < 0:
        print("ERROR: INVALID VALUE")

    elif SCADA_second_input.get() >= 0:
        SCADA_alarm2 = SCADA_second_input.get()
        print("BMS Alarm 1 set to:", SCADA_alarm2)

def alarm_check1():
    global BMS_alarm1, log_status
    if BMS_alarm1 is not None:
        current_value = read_channel()
        if current_value > BMS_alarm1 and not log_status:
            log_status = True
            threading.Thread(target=log_data, daemon=True).start()
            GPIO.output(Relay1, GPIO.HIGH)
        elif current_value <= BMS_alarm1 and log_status:
            log_status = False
            GPIO.output(Relay1, GPIO.LOW)
    app.after(300, alarm_check1)

def alarm_check2():
    global BMS_alarm2, log_status
    if BMS_alarm2 is not None:
        current_value = read_channel()
        if current_value > BMS_alarm2 and log_status:
            log_status = False
            GPIO.output(Relay2, GPIO.HIGH)
        elif current_value <= BMS_alarm2:
            GPIO.output(Relay2, GPIO.LOW)
    app.after(300, alarm_check2)


def alarm_check3():
    global SCADA_alarm1
    if SCADA_alarm1 is not None:
        current_value = read_channel()
        if current_value > BMS_alarm1:
            GPIO.output(Relay3, GPIO.HIGH)
        else:
            GPIO.output(Relay3, GPIO.LOW)
    app.after(300, alarm_check3)

def alarm_check4():
    global SCADA_alarm2
    if SCADA_alarm2 is not None:
        current_value = read_channel()
        if current_value > BMS_alarm2:
            GPIO.output(Relay4, GPIO.HIGH)
        else:
            GPIO.output(Relay4, GPIO.LOW)
    app.after(300, alarm_check4)

def functional_test():
    print("Performing system functionality test....")
    GPIO.output(Relay1, GPIO.HIGH)
    time.sleep(1)
    GPIO.output(Relay1, GPIO.LOW)
    GPIO.output(Relay2, GPIO.HIGH)
    time.sleep(1)
    GPIO.output(Relay2, GPIO.LOW)
    GPIO.output(Relay3, GPIO.HIGH)
    time.sleep(1)
    GPIO.output(Relay3, GPIO.LOW)
    GPIO.output(Relay4, GPIO.HIGH)
    time.sleep(1)
    GPIO.output(Relay4, GPIO.LOW)

def read_channel():
    global current
    adc = spi.xfer2([1, (8 + 0) << 4, 0])
    raw_value = ((adc[1] & 3) << 8) + adc[2]
    
    min_adc = 310  # berekening aanpassen
    max_adc = 1023  
    max_current = 100
    if raw_value >= min_adc:
        normalized_value = (raw_value - min_adc) / (max_adc - min_adc)  
        current = normalized_value * max_current
        print("The current value:", current)
    else:
        current = 0
    
    return current

def update_current_display():
    current_value = read_channel()
    current_label.configure(text=f"Current: {current_value:.2f} A")
    app.after(500, update_current_display)  


def append_to_input(value):
    if focused_entry:
        current_text = focused_entry.get()
        focused_entry.delete(0, tkinter.END)
        focused_entry.insert(0, current_text + value)


def backspace_input():
    if focused_entry:
        current_text = focused_entry.get()
        focused_entry.delete(0, tkinter.END)
        focused_entry.insert(0, current_text[:-1])

def log_data():
    global current
    print("Starting data logging...")
    with open("/home/ret/Desktop/App/log_report.csv", "a") as log:
        while log_status:
            log.write("{0},{1}\n".format(strftime("%Y-%m-%d %H:%M:%S"), float(current)))
            sleep(0.3)
    print("Data logging stopped.")

app = customtkinter.CTk()
app.geometry("800x480")
app.title("RET Demo Dashboard")
app.grid_columnconfigure(0, weight=1)
app.grid_columnconfigure(1, weight=1)


customtkinter.CTkLabel(app, text="Set First BMS alarm").grid(row=0, column=3, padx=30, pady=0)
customtkinter.CTkLabel(app, text="Set second BMS alarm").grid(row=2, column=3, padx=30, pady=0)

BMS_first_input = tkinter.DoubleVar()
input1 = customtkinter.CTkEntry(app, width=100, height=30, textvariable=BMS_first_input)
input1.grid(row=1, column=3, padx=20, pady=0)
input1.bind("<FocusIn>", lambda e: on_focus(input1))

BMS_second_input = tkinter.DoubleVar()
input2 = customtkinter.CTkEntry(app, width=100, height=30, textvariable=BMS_second_input)
input2.grid(row=3, column=3, padx=20, pady=0)
input2.bind("<FocusIn>", lambda e: on_focus(input2))

customtkinter.CTkButton(app, width=60, height=30, text="SET", command=BMS_set1).grid(row=1, column=5, padx=20, pady=0)
customtkinter.CTkButton(app, width=60, height=30, text="SET", command=BMS_set2).grid(row=3, column=5, padx=20, pady=0)

customtkinter.CTkLabel(app, text="Set First SCADA alarm").grid(row=0, column=1, padx=30, pady=0)
customtkinter.CTkLabel(app, text="Set second SCADA alarm").grid(row=2, column=1, padx=30, pady=0)

SCADA_first_input = tkinter.DoubleVar()
input3 = customtkinter.CTkEntry(app, width=100, height=30, textvariable=SCADA_first_input)
input3.grid(row=1, column=1, padx=20, pady=0)
input3.bind("<FocusIn>", lambda e: on_focus(input3))

SCADA_second_input = tkinter.DoubleVar()
input4 = customtkinter.CTkEntry(app, width=100, height=30, textvariable=SCADA_second_input)
input4.grid(row=3, column=1, padx=20, pady=0)
input4.bind("<FocusIn>", lambda e: on_focus(input4))

customtkinter.CTkButton(app, width=60, height=30, text="SET", command=SCADA_set1).grid(row=1, column=2, padx=20, pady=0)
customtkinter.CTkButton(app, width=60, height=30, text="SET", command=SCADA_set2).grid(row=3, column=2, padx=20, pady=0)


customtkinter.CTkButton(app, width=150, height=30, text="Full system test", command=functional_test).grid(row=1, column=0, padx=20, pady=10)


current_label = customtkinter.CTkLabel(app, text="Current: 0.00 A", font=("Arial", 24))
current_label.grid(row=5, column=3, columnspan=3, pady=20)


numpad_frame = customtkinter.CTkFrame(app, width=200, height=200)
numpad_frame.place(relx=1.0, rely=1.0, anchor="se")

buttons = [
    ('1', 0, 0), ('2', 0, 1), ('3', 0, 2),
    ('4', 1, 0), ('5', 1, 1), ('6', 1, 2),
    ('7', 2, 0), ('8', 2, 1), ('9', 2, 2),
    ('.', 3, 0), ('0', 3, 1), ('⌫', 3, 2)
]

for (text, row, col) in buttons:
    if text == '⌫':
        btn = customtkinter.CTkButton(numpad_frame, text=text, command=backspace_input)
    else:
        btn = customtkinter.CTkButton(numpad_frame, text=text, command=lambda t=text: append_to_input(t))
    btn.grid(row=row, column=col, padx=5, pady=5, sticky="nsew")

numpad_frame.grid_rowconfigure((0, 1, 2, 3), weight=1)
numpad_frame.grid_columnconfigure((0, 1, 2), weight=1)


update_current_display()
alarm_check1()
alarm_check2()
alarm_check3()
alarm_check4()



app.mainloop()